# Clean NCES schools

Transforms historic lists of schools downloaded from federal database. 

In [1]:
import os
import pandas as pd

In [2]:
import warnings
warnings.simplefilter("ignore")

In [3]:
%store -r

For cleaning up address columns. Works for all federal and local school files.

In [4]:
replace = [
    ['W.', 'West'],
    ['E.', 'East'],
    ['S.', 'South'],
    ['N.', 'North'],
    ['Pkwy', 'Parkway'],
    ['Place', 'Pl'],
    ['Pla', 'Pl'],
    ['Avenue', 'Ave'],
    ['Broadway', 'BrdWay'],
    ['  ',' '],
    [',',''],
    ['.','']
]

In [5]:
def clean_addy(street):
    """
    Submit a dirty address and get a clean one back.
    """
    for key, value in replace:
        street = street.replace(key, value)
    return street

Functions to clean up school files. First a generic one for most files.

In [15]:
def transform(filename, filetype):
    """
    Reads in file for each year and creates school code column
    """
    #Read in differently depending on file type
    path = os.path.join(input_dir, filename)
    if filetype == 'csv':
        df = pd.read_csv(path, encoding="latin-1")
    elif filetype == 'txt':
        df = pd.read_csv(path, delimiter='\t', encoding="latin-1")
    
    #Filter out to California
    cal_df = df[df.STATENAME == 'CALIFORNIA']
    
    #Filter out to LA county  
    cal_df['county'] = cal_df.ST_LEAID.apply(lambda x: str(x)[-7:-5])
    cal_df = cal_df[cal_df.county == '19']
    
    #Create column for school code. Adds leading zero where necessary.
    cal_df['school_code'] = cal_df['ST_SCHID'].apply(lambda x: x[-7:] if type(x)==str else str('{:07d}'.format(x)))
    
    #Trim to three columns 
    cal_df_trim = cal_df[['school_code', 'LSTREET1', 'LCITY']]
    
    #Pretty columns
    cal_df_trim.rename(columns={
        'LSTREET1': 'street',
        'LCITY': 'city',
    }, inplace=True)
    
    #Everything to title case for matching later
    cal_df_clean = cal_df_trim.apply(lambda x: x.astype(str).str.title()) 
    
    #Cleaning up street column for matching later
    cal_df_clean['street'] = cal_df_clean['street'].apply(clean_addy)
        
    return cal_df_clean

Now a special one for the oddball file from 2013-14.

In [12]:
def transform_1314(filename):
    """
    Transforms 2013-2014 file from NCES
    """
    df = pd.read_csv(os.path.join(input_dir, filename), delimiter='\t')
    
    #Filter to CA
    df = df[df.FIPST == 6] 
    
    #Filter to LA county
    df['county'] = df.STID.apply(lambda x: str(x)[-7:-5])
    df = df[df.county == '19']
    
    df.rename(columns={
        'SEASCH':'school_code',
        'LSTREE':'street', 
        'LCITY':'city',
    }, inplace=True)
    df = df[['school_code','street','city']]

    df['school_code'] = df['school_code'].apply(lambda x: str('{:07d}'.format(x)))
    
    df_new = df.apply(lambda x: x.astype(str).str.title()) 
    
    df_new['street'] = df_new['street'].apply(clean_addy)
    
    return df_new

Transform national files.

In [8]:
df1314 = transform_1314('sc132a.txt')

In [9]:
df1415  = transform('ccd_sch_029_1415_w_0216601a.txt', 'txt')

In [16]:
df1516  = transform('ccd_sch_029_1516_w_2a_011717.csv', 'csv')

In [17]:
df1617  = transform('ccd_sch_029_1617_w_1a_11212017.csv', 'csv')

Output files

In [23]:
write_df = lambda df, year: df.to_csv(os.path.join(output_dir, "df_{}.csv".format(year)), index=False, encoding="utf-8")

In [24]:
write_df(df1314, "1314")

In [25]:
write_df(df1415, "1415")

In [26]:
write_df(df1516, "1516")

In [27]:
write_df(df1617, "1617")